# Analyze fitness effects of mutations fixed in each clade

Import Python modules:

In [1]:
import altair as alt

import pandas as pd

import yaml

_ = alt.data_transformers.disable_max_rows()

Now get variables from `snakemake`:

In [2]:
if "snakemake" not in globals() and "snakemake" not in locals():
    # variables set manually for interactive debugging
    aa_fitness_csv = "../results/aa_fitness/aa_fitness.csv"
    aamut_by_clade_csv = "../results/aa_fitness/aamut_fitness_by_clade.csv"
    clade_founder_nts_csv = "../results/clade_founder_nts/clade_founder_nts.csv"
    
    with open("../config.yaml") as f:
        config = yaml.safe_load(f)
    min_expected_count = config["min_expected_count"]
    ref = config["clade_fixed_muts_ref"]
    
else:
    # get variables from `snakemake` when running pipeline
    aa_fitness_csv = snakemake.input.aafitness
    aamut_by_clade_csv = snakemake.input.aamut_by_clade
    clade_founder_nts_csv = snakemake.input.clade_founder_nts_csv
    min_expected_count = snakemake.params.min_expected_count
    ref = snakemake.params.ref

Read the input data:

In [3]:
aa_fitness = pd.read_csv(aa_fitness_csv)
aamut_by_clade = pd.read_csv(aamut_by_clade_csv)
clade_founder_nts = pd.read_csv(clade_founder_nts_csv)